## Example 04 - Building a Layered Dike

**Example description:** Example of a construction challenge, with four separate activites, where each activity depends on the progress of the other activities.

* [**0. Import required libraries:**](#0.-Import-required-libraries)<br>
* [**1. Define work method:**](#1.-Define-work-method)<br>
   * [**1.1 Sites:**](#1.1-Define-the-project-sites)<br>
   * [**1.2 Equipment:**](#1.2-Define-the-project-equipment)<br>
   * [**1.3 Activities:**](#1.3-Define-the-activity)<br>
* [**2. Run the simulation:**](#2.-Run-the-simulation)<br>
* [**3. Post processing:**](#3.-Postprocessing)<br>

## 0. Import required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import pandas as pd

# digital twin package
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

In [2]:
# Create simulation environment
simulation_start = datetime.datetime.now()

my_env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
my_env.epoch = time.mktime(simulation_start.timetuple())

## 1. Define work method

### 1.1 Define the project sites
You can specify a project site object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.Locatable - enables you to add coordinates to extract distance information and visualize
    core.HasContainer - enables you to add information on the material available at the site
    core.HasResource - enables you to add information on serving equipment
    core.HasWeather - enables you to add weather conditions

#### First create objects with the desired properties

In [3]:
Site = type('Site', (core.Identifiable, 
                     core.Log, 
                     core.Locatable, 
                     core.HasContainer, 
                     core.HasResource), {})

#### Next specify the properties for sites you wish to create

In [4]:
# origin sites data
data_stock_clay = {"env": my_env,
                   "name": "Clay_Stock",
                   "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}

data_stock_sand = {"env": my_env,
                   "name": "Sand_Stock",
                   "geometry": shapely.geometry.Point(5.271417603333632, 52.9638452897506),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}

data_stock_armour = {"env": my_env,
                   "name": "Armour_Stock",
                   "geometry": shapely.geometry.Point(5.919298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}

data_stock_levvel = {"env": my_env,
                   "name": "Levvel_Stock",
                   "geometry": shapely.geometry.Point(5.919298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}

In [5]:
# destination sites data

# nr of bins to use
site_count = 20

data_clay = {"start": [5.054676856441372, 52.94042293840172], # Den Oever (lon, lat)
             "stop": [5.294877712236641, 53.06686424241725], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Clay_Layer"}

data_sand = {"start": [5.052051052879287,52.9421894472733], # Den Oever (lon, lat)
             "stop": [5.292216781509101,53.06886359869087], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Sand_Layer"}

data_armour = {"start": [5.049510554598302,52.94393628899332], # Den Oever (lon, lat)
             "stop": [5.289636346490858,53.07053144816584], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Armour_Layer"}

data_levvel = {"start": [5.046556507026805,52.94579445406793], # Den Oever (lon, lat)
             "stop": [5.286775240694118,53.07264015015531], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Levvel_Layer"}

#### Finally create specific instances of the predefined objects with the specified properties

In [6]:
# create the site objects
stocks = []
for data in [data_stock_clay, data_stock_sand, data_stock_armour, data_stock_levvel]:
    stock = Site(**data)
    stocks.append(stock)

In [7]:
sites = []
for data_layer in [data_clay, data_sand, data_armour, data_levvel]:
    start = data_layer['start']
    stop = data_layer['stop']

    # generate the evenly spaced locations between the selected start and stop points
    lons = np.linspace(start[0], stop[0], num=site_count)
    lats = np.linspace(start[1], stop[1], num=site_count)

    # create a Site object for each location
    layer_sites = []
    for i in range(site_count):
        data_site = {"env": my_env,
                     "name": data_layer["name"] + '_' + format(i, '02.0f'),
                     "geometry": shapely.geometry.Point(lons[i], lats[i]),
                     "capacity": data_layer["capacity"]}
        site = Site(**data_site)
        layer_sites.append(site)
    sites.append(layer_sites)

### 1.2 Define the project equipment
You can specify a vessel object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.ContainerDependentMovable - A moving container, so capacity and location
    core.Processor - Allow for loading and unloading
    core.HasResource - Add information on serving equipment
    core.HasDepthRestriction - Add information on depth restriction 

#### First create objects with the desired properties

In [8]:
TransportResource = type('TransportResource', 
                         (core.Identifiable, 
                          core.Log, 
                          core.ContainerDependentMovable, 
                          core.HasResource), {})

TransportProcessingResource = type('TransportProcessingResource', 
                                   (core.Identifiable, 
                                    core.Log, 
                                    core.ContainerDependentMovable, 
                                    core.Processor, 
                                    core.LoadingFunction,
                                    core.UnloadingFunction,
                                    core.HasResource), {})

ProcessingResource = type('ProcessingResource', 
                          (core.Identifiable, 
                           core.Log, 
                           core.Processor, 
                           core.LoadingFunction,
                           core.UnloadingFunction,
                           core.HasResource,
                           core.Movable), {})

#### Next specify the properties for equipment you wish to create

In [9]:
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty

In [10]:
# equipment information in database
data_gantry_crane = {"env": my_env,
                     "name": "Gantry crane",
                     "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                     "loading_rate": 0.1,                                   # Loading rate
                     "unloading_rate": 0.1}                                 # Unloading rate


data_installation_crane = {"env": my_env,
                           "name": "Installation crane",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "loading_rate": 0.5,                                   # Loading rate
                           "unloading_rate": 0.5}                                 # Unloading rate


data_transport_barge_01 = {"env": my_env,
                           "name": "Transport barge 01",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "capacity": 1_000,
                           "compute_v": compute_v_provider(v_empty=1.6, v_full=1)}

data_transport_barge_02 = {"env": my_env,
                           "name": "Transport barge 02",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "capacity": 1_000,
                           "compute_v": compute_v_provider(v_empty=1.6, v_full=1)}

# TSHD variables
data_hopper = {"env": my_env,                                       # The simpy environment 
               "name": "Hopper",                                    # Name
               "geometry": shapely.geometry.Point(5.070195628786471, 52.93917167503315),                      # It starts at the "from site"
               "loading_rate": 1.5,                                   # Loading rate
               "unloading_rate": 1.5,                                 # Unloading rate
               "capacity": 1_000,                                   # Capacity of the hopper - "Beunvolume"
               "compute_v": compute_v_provider(2, 1.5)}             # Variable speed 

#### Finally create specific instances of the predefined objects with the specified properties

In [11]:
# create the processing resources
gantry_crane = ProcessingResource(**data_gantry_crane)
installation_crane = ProcessingResource(**data_installation_crane)

# create the transport resources
transport_barge_01 = TransportResource(**data_transport_barge_01)
transport_barge_02 = TransportResource(**data_transport_barge_02)

# create the transport processing resource
hopper = TransportProcessingResource(**data_hopper)

### 1.3 Define the activity
As can be seen in the code fragment above, the Activity class takes the Simpy environment and a name as its constructor arguments. This is because it is defined as a mixin of the SimpyObject and Identifiable core classes. To construct an activity we also needed to define the origin, destination, loader, mover and unloader. Finally, we can optionally define conditions that need to be satisfied for the transportation of materials to take place. These are separated in __start_condition__, __stop_condition__ and __condition__ arguments:
* __start_condition__: The activity will start as soon as this condition is satisfied. If this condition is not yet satisfied at the start of the simulation the activity will be suspended untill the condition is satisfied at a later time. Since we do not want any clay to be transported to the later sites if the previous site has not yet been completed, we define this condition as a start_condition.
* __stop_condition__: After the activity has started (the start_condition was satisfied), it will regularly check if the stop_condition has been satisfied. If so, the activity is considered complete and will terminate. A simulation will not terminate untill all of its activities have been completed. By default (with stop_condition=None), the stop_condition is set to either the origin container being empty, or the destination container being full. This is sufficient in our case, so we do not specify a stop_condition. 
* __condition__: After the activity has started (the start_condition was satisfied) and as long as the stop_condition is not satisfied, the activity will continuously check whether this condition is satisfied. If so, it will complete exactly one transportation of materials (loading the mover at the origin and unloading the mover at the destination). If not, the activity will wait untill the condition is satisfied again. Defining such a condition allows for the activity to be temporarily halted, for example due to bad weather or other working conditions, and resuming at a later time when the problem preventing work has passed. 

All of these condition arguments take a "Condition" object. This object should have a "satisfied" method which returns a boolean indicating whether the condition is satsified or not. The digital twin package provides a LevelCondition, allowing you to quickly construct a condition which will check if an object extending HasContainer has a certain minimum and/or maximum level. It also provides an AndCondition and OrCondition which allow for combining other condition objects into a single condition. You can also provide your own "Condition" class, for example you could program a "Condition" class which depending on the time in the simulation (the .now fixture of the Simpy environment), returns whether the weather is good enough for work to take place:

In [12]:
activities = []

In [13]:
# define the sand layer Activities
first_clay_activity = len(activities)
clay_layer_index = 0
clay_sites = sites[clay_layer_index]
for i, clay_site in enumerate(clay_sites):
    for barge_index, transport_barge in enumerate([transport_barge_01, transport_barge_02]):
        # for the first site we do not need to define any condition
        activity = model.Activity(env=my_env,
                                  name="Clay_Placement_S" + format(i+1, '02.0f') + "_B" + format(barge_index + 1, '02.0f'),
                                  origin=stocks[clay_layer_index],
                                  destination=clay_site,
                                  loader=gantry_crane,
                                  mover=transport_barge,
                                  unloader=installation_crane)
        activities.append(activity)

In [14]:
# define the sand layer Activities
first_sand_activity = len(activities)
sand_layer_index = 1
sand_sites = sites[sand_layer_index]
for i, sand_site in enumerate(sand_sites):
    # The sand layer activities can start as soon as the clay is placed and 
    # the adjoining sand layer is finished
    start_event = activities[i + first_clay_activity].main_process if i == 0 else \
                  [activities[i + first_clay_activity].main_process, activities[-1].main_process]

    activity = model.Activity(env=my_env,
                              name="Sand_Placement_S" + format(i+1, '02.0f'),
                              origin=stocks[sand_layer_index],
                              destination=sand_site,
                              loader=hopper,
                              mover=hopper,
                              unloader=hopper,
                              start_event=start_event)
    activities.append(activity)

In [15]:
# define the armour and level layer activities
first_armour_activity = len(activities)
layer_names = ["Armour_Placement", "Levvel_Placement"]
layer_indices = [2, 3]

for layer_index in layer_indices:
    i = 0 if layer_index == 2 else 0
    
    for barge_index, transport_barge in enumerate([transport_barge_01, transport_barge_02]):
        layer_sites = sites[layer_index]
        for site_index, site in enumerate(layer_sites):
            # The first armour layer activities can start as soon as the sand is placed and 
            # the adjoining armour layer is finished
            if layer_index == 2:
                start_event = activities[i + first_sand_activity].main_process if i == 0 else \
                              [activities[i + first_sand_activity].main_process, activities[-1].main_process]
            elif layer_index == 3:
                start_event = activities[i + first_armour_activity].main_process if i == 0 else \
                              [activities[i + first_armour_activity].main_process, activities[-1].main_process]
            
            # for the first site we do not need to define a condition on the previous site
            activity = model.Activity(env=my_env,
                                      name=layer_names[layer_index-2] + "_S" + format(site_index+1, '02.0f')
                                           + "_B" + format(barge_index+1, '02.0f'),
                                      origin=stocks[layer_index],
                                      destination=site,
                                      loader=gantry_crane,
                                      mover=transport_barge,
                                      unloader=installation_crane,
                                      start_event=start_event)
            
            activities.append(activity)
            i += 1

### 2. Run the simulation

In [16]:
my_env.run()

print("\n*** Activity finished in {} ***".format(datetime.timedelta(seconds=int(my_env.now - my_env.epoch))))


*** Activity finished in 247 days, 20:42:18 ***


### 3. Postprocessing

In [17]:
pd.DataFrame.from_dict(transport_barge_01.log).tail()

,Message,Timestamp,Value,Geometry,ActivityID
2073,loading stop,2020-04-27 00:09:15.362486,1000.0,POINT (5.919298185633251 52.94239823421129),364daf48-c5b2-11e9-b6e3-b469212bff5b
2074,sailing filled start,2020-04-27 00:09:15.362486,1000.0,POINT (5.919298185633251 52.94239823421129),364daf48-c5b2-11e9-b6e3-b469212bff5b
2075,sailing filled stop,2020-04-27 12:36:59.431103,1000.0,POINT (5.286775240694118 53.07264015015531),364daf48-c5b2-11e9-b6e3-b469212bff5b
2076,unloading start,2020-04-27 12:36:59.431103,0.0,POINT (5.286775240694118 53.07264015015531),364daf48-c5b2-11e9-b6e3-b469212bff5b
2077,unloading stop,2020-04-27 13:10:19.431103,1000.0,POINT (5.286775240694118 53.07264015015531),364daf48-c5b2-11e9-b6e3-b469212bff5b


In [18]:
pd.DataFrame.from_dict(transport_barge_02.log).tail()

,Message,Timestamp,Value,Geometry,ActivityID
313,loading stop,2019-09-08 03:31:25.077671,1000.0,POINT (5.019298185633251 52.94239823421129),36471f87-c5b2-11e9-b60a-b469212bff5b
314,sailing filled start,2019-09-08 03:31:25.077671,1000.0,POINT (5.019298185633251 52.94239823421129),36471f87-c5b2-11e9-b60a-b469212bff5b
315,sailing filled stop,2019-09-08 09:56:35.611195,1000.0,POINT (5.294877712236641 53.06686424241725),36471f87-c5b2-11e9-b60a-b469212bff5b
316,unloading start,2019-09-08 09:56:35.611195,0.0,POINT (5.294877712236641 53.06686424241725),36471f87-c5b2-11e9-b60a-b469212bff5b
317,unloading stop,2019-09-08 10:29:55.611195,1000.0,POINT (5.294877712236641 53.06686424241725),36471f87-c5b2-11e9-b60a-b469212bff5b


#### Vessel planning

In [19]:
vessels = [gantry_crane, 
           installation_crane, 
           transport_barge_01, 
           transport_barge_02, 
           hopper]

activities = ['loading', 'unloading', 'sailing filled', 'sailing empty', 'processing']
colors = {0:'rgb(55,126,184)', 1:'rgb(255,150,0)', 2:'rgb(98, 192, 122)', 3:'rgb(98, 141, 122)', 4:'rgb(110, 141, 122)'}

plot.vessel_planning(vessels, activities, colors)

#### KML visualisation

In [20]:
plot.vessel_kml(my_env, vessels[2:3], stepsize = 3600)

In [21]:
# open the file
if platform.system():
    !start ./vessel_movements.kml
else:
    !start explorer ./vessel_movements.kml